In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def cgls_early_stopping_regularization_with_invertible_transformation(A, R, b, noise_sigma, maxits=100, tau=1.03, early_stopping=True):
    """Performs CGLS with early stopping regularization applied to minimizing
                || A x - b ||_2 ,
    stopping at the first iteration such that
        || A x - b ||_2^2 <= tau * m * noise_var.
    We assume that R is a given invertible square matrix.
    """

    n = A.shape[1]
    m = A.shape[0]
    x = np.zeros(n)
    r_prev = b - (A @ x)
    d_prev = A.T @ r_prev

    # Tracking
    squared_residuals = []
    squared_res = np.linalg.norm((A @ x) - b)**2
    squared_residuals.append(squared_res)
    n_iterations = 0

    for k in range(maxits):

        # CGLS iteration
        alpha = (np.linalg.norm(A.T @ r_prev)**2)/(np.linalg.norm(A @ d_prev)**2)
        x = x + alpha*d_prev
        r_next = r_prev - alpha*(A @ d_prev)
        beta = (np.linalg.norm(A.T @ r_next)**2/(np.linalg.norm(A.T @ r_prev)**2))
        d_next = (A.T @ r_next) + beta*d_prev

        d_prev = d_next
        r_prev = r_next

        # Track this
        n_iterations += 1
        squared_res = np.linalg.norm((A @ x) - b)**2
        squared_residuals.append(squared_res)

        if early_stopping and (squared_res < tau*(m*(noise_sigma**2))):
            break

    data = {
        "x": x,
        "n_iterations": n_iterations,
        "squared_residuals": squared_residuals,
    }

    return data